### Import Libraries

In [20]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Dense, BatchNormalization, Concatenate, Activation, MaxPool2D, AveragePooling2D, GlobalAveragePooling2D, Input, Dropout, Convolution2D, AvgPool2D
from tensorflow.keras.models import Model

### Conv block

In [27]:
def bn_relu_conv(prev_layer, filters, kernel_size=1):
  """
  Applies BN -> relu -> conv in order

  parameters:
    prev_layer (tensor) -> input layer
    filters(int) -> the number of filters for convolution layer
    kernel_size(int) -> sliding window kernel_size of convolution layer

  returns:
    x -> tensor
  """
  x = BatchNormalization()(prev_layer)
  x = Activation(activation='relu')(x)
  x = Conv2D(filters, kernel_size, padding='same')(x)
  return x

### Dense block

In [28]:
def dense_block(prev_layer, filters, repetitions, dropout_rate=None):
  """
  Feature conatenation is performed in Dense block where each layer is connected to every layer in the block
  Applies 1x1 conv -> 3x3 conv -> concat

  parameters:
    prev_layer (tensor) -> input layer
    filters(int) -> the number of filters for convolution layer(based on growth rate in densenet)
    repetitions(int) -> no of times a block should be repeated
    dropout_rate -> enable dropout

  returns:
    x -> tensor
  """
  for i in range(repetitions):
    x = bn_relu_conv(prev_layer, filters = 4*filters, kernel_size=1)
    if dropout_rate:
      x = Dropout(rate=dropout_rate)(x)
    x = bn_relu_conv(x, filters=filters, kernel_size=3)
    prev_layer = Concatenate(axis=-1)([prev_layer, x])
  return prev_layer


### Transition block

In [36]:
def transition_block(x , theta=0.5):
  """
  Dimensionality reduction is performed in this block
  Applies 1x1 conv -> Avgpool

  (0 < theta <= 1) if a dense block contains M feature-maps , we let the following transition layer generate (thete of M) output feature-maps

  parameters:
    x (tensor) -> input layer
    theta(float) -> compression rate

  returns:
    x -> tensor

  """
  f = int(tf.keras.backend.int_shape(x)[-1] * theta) # prev_layer_channel_shape/2
  x = bn_relu_conv(x , filters=f , kernel_size=1)
  x = AvgPool2D(pool_size=2 , strides=2 , padding="same")(x)
  return x

### DenseNet121 Model

In [38]:
def DenseNet121(input_shape, num_classes, growth_rate, repeatations):
  """
  DenseNet121 Architecture

  Parameters:
    input_shape - input shape to model (H,W,C)
    num_classes - Num of classes to classify (Eg:1000 for Imagenet)
    growth_rate  - Number of filters to be applied to each layer
    repeatations - Number of times a dense block should be repeated

  returns:
    model : tf densenet model
  """
  input = Input(shape=input_shape)

  x = Conv2D(filters=2*growth_rate, kernel_size=(7,7), strides=2, padding='same')(input)
  x = MaxPool2D(pool_size=(3,3), strides=2, padding='same')(x)

  for i in repeatations:
    d = dense_block(x, filters=growth_rate, repetitions=i)
    x = transition_block(d, theta=0.5)
  x = GlobalAveragePooling2D()(d)
  output = Dense(num_classes, activation = 'softmax')(x)

  model = Model(input, output)
  return model

### Model summary

In [39]:
k = 32
REPEATATIONS = [6,12,24,16]
INPUT_SHAPE = (224,224,3)
n_classes = 1000

model = DenseNet121(input_shape=INPUT_SHAPE, num_classes=n_classes, growth_rate = k, repeatations=REPEATATIONS)

In [33]:
from tensorflow.keras.utils import plot_model
plot_model(model , show_shapes=True)

Output hidden; open in https://colab.research.google.com to view.